# Open a microscopy data in any format 

This notebook enables the user to load microscopy data in different formats listed [here](https://bio-formats.readthedocs.io/en/v6.12.0/supported-formats.html) in a similar fashion. It also enables the reading of the metadata. The installation of the package aicsimageio is required. Documentation can be found [here](https://allencellmodeling.github.io/aicsimageio/index.html).
The following steps are required for the installation of the required package: <br>
> \$ conda activate \<env\> <br>
    \$ pip install aicsimageio <br>
    \$ pip install aicsimageio[base-imageio,nd2,bfio,all] <br>
    \$ pip install aicsimageio readlif>=0.6.4 <br>
    \$ pip install aicsimageio aicspylibczi>=3.0.5 fsspec>=2022.7.1 <br>
    \$ conda install -c conda-forge bioformats_jar <br> 

Load the image with its metadata in this next cell by replacing the image path directory in that cell. Make sure to have the path correct and use / instead of \

In [1]:
# EXAMPLE
from scyjava import config
from aicsimageio import AICSImage
# mydata = AICSImage("Y:/April05/FluoCells1_BPAE_F36924/FluoCells1_F36924_001.nd2") # THE IMAGE PATH DIRECTORY HERE
mydata=AICSImage("C:/Users/Dina Ratsimandresy/Documents/MRC-Office/Dataset/PythonCourse precious images/Airy_4color_40nm_67e5micron_diversoformato-confocal.tif")
# mydata=AICSImage("Y:/April05/TS_10_2_dillution_ProlongGlass/TS_100nm_ProlongGlass/hCZT_ch00.tif")

The following packages need to be imported to display the image and extract the metadata from the data.

In [2]:
# IMPORT PACKAGES

from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.cm import ScalarMappable
from matplotlib.colors import Normalize
import matplotlib.colors as mcolors
from matplotlib.colors  import LinearSegmentedColormap, ListedColormap

Next, we read the image and the metadata. Metadata includes the number of channels in the images, the name of each channel and emission wavelength associated with each channel, the voxel size and physical size of the image. Many other acquisition parameters are stored in the metadata and we can access them from here.

In [3]:
# READ IMAGE AND THE METADATA

metadata = mydata.metadata 
myimg = mydata.data
imgsize = myimg.shape

if type(metadata)==dict:
    metadata = mydata.metadata
    channumber=len(metadata['metadata'].channels) # or simply imgsize[1]
    channelname=[metadata['metadata'].channels[c].channel.name for c in np.arange(channumber)] # 
    wavelengthlist=[metadata['metadata'].channels[c].channel.emissionLambdaNm for c in np.arange(channumber)] # emission wavelength in each channel
    voxelsize = metadata['metadata'].channels[0].volume.axesCalibration # voxel size in um
    voxelcount = metadata['metadata'].channels[0].volume.voxelCount # 3D size
    NA = metadata['metadata'].channels[0].microscope.objectiveNumericalAperture
    ri_immersion = metadata['metadata'].channels[0].microscope.immersionRefractiveIndex # refractive index of the immersion medium 
    microscope_name = metadata['metadata'].channels[0].microscope.objectiveName
    Acquisition = metadata['text_info']['capturing']  # contains info about the exposure time of each channel 
    # print(Acquisition)
    print('Data recorded from:', microscope_name)
    print('Time frames:', imgsize[0])
    print('Voxel size:', voxelsize,'in micrometer')

    df = pd.DataFrame(columns=channelname,dtype=object)
    data = {channelname[c]:wavelengthlist[c] for c in np.arange(channumber)}
    df = pd.DataFrame(data=data,index=['Emission wavelength [in nm]'])
    df
elif type(metadata)==str:
    channumber = imgsize[1]
    channelname = ['Channel-'+str(c) for c in np.arange(channumber)]
    voxelcount = imgsize[-1:1:-1]
print('Size XYZ of the image:', voxelcount)
print('Number of channels:', channumber)

Size XYZ of the image: (1912, 1912, 1)
Number of channels: 4


The image is usually in 5D where the first three dimensions are spatials, the channel and timeframe are in the fourth and fifth dimension respectively. In the next cell, we generate a list of colormaps for displaying each channel. Users are advised to choose their desired colormaps based on their data and number of channels in the data. Otherwise we assume the first four existing channels are associated with RGGB (red, green, grey, blue) colors. If the data contains more than the number of colors mentioned in the dictionary below, another modification of the notebook is required.

In [4]:
# COLORMAP FOR DISPLAYING DIFFERENT CHANNELS 
colorslist = {'red':(1,0,0),'yellow':(1,1,0), 'green':(0,1,0),'cyan':(0,1,1),'grey':(1,1,1),'pink':(1,0,1),'blue':(0,0,1)} # a dictionary of colors
cmpsuggestion=['red','green','grey','blue'] # list of colormaps to be applied on the channels, choose the color per order of the channels of your data here!

The image is displayed using the code in the next cell. The display is interactive where the user can select the channel to display and slide over the axial (z) position and time frame in the data. The combination of all the channels is displayed below the display of the selected channel.

In [5]:
# DISPLAY THE IMAGES INTERACTIVELY

def series(time, zpos, channel):

    shp = (myimg.shape[3],myimg.shape[4],channumber) # tuple
    rgb = np.zeros(shp,dtype=myimg.dtype)

    for k in np.arange(channumber):
        rgb[:,:,k] = myimg[time,k,zpos,:,:]

    lb=0; # lower bound equal to 0
    if lb==0:
        s=1
    else:
        s=-np.sign(lb) # this should not be an option since we are dealing with intensity so the lower bound in the original data should always be positive but for future use, keep it as it is!
    ub=1; # upper bound equal to 1

    maxValue = np.expand_dims(np.array(np.amax(np.amax(rgb,axis=0),axis=0)),axis=[0,1])
    minValue = np.expand_dims(np.array(np.amin(np.amin(rgb,axis=0),axis=0)),axis=[0,1])
    rgbupd=(rgb-minValue)/(maxValue - minValue)*(s*np.abs(lb)+np.abs(ub))-s*np.abs(lb) # this normalization is needed for display using the colormap and for making a stack of all channels

    cmappable_list={}

    for k in range(channumber):
        colorsrange = [(0, 0, 0), colorslist[cmpsuggestion[k]]] # first color is black, last is red
        cmpk = LinearSegmentedColormap.from_list(cmpsuggestion[k], colorsrange, N=256)
        colors = cmpk(np.arange(0,cmpk.N)) 
        color_matplotlib = ListedColormap(colors)
        chank = color_matplotlib(rgbupd[:,:,k])
        cmappable = ScalarMappable(norm=Normalize(minValue[0][0][k],maxValue[0][0][k]), cmap=cmpk) # display of each channel with its original pixel range here

        if k in cmappable_list:
            cmappable_list[k].append(cmappable)
        else:
            cmappable_list[k] = cmappable # save the colormap to display the channel

        chank = np.expand_dims(chank,axis=3)
        if k==0:
            rgb = chank
        else:
            rgb = np.append(rgb,chank,axis=3)

    rgb_proj = np.max(rgb, axis = 3) # all the channels combined 

    # Dislay single channel
    channels = {channelname[c]:c for c in np.arange(channumber)}

    fig, ax = plt.subplots()
    cpos=channels[channel]
    ax.imshow(rgb[:,:,:,cpos])
    fig.colorbar(cmappable_list[cpos])


    # Display RGB
    print(channumber,'channels combined at the given timeframe t =',time,'and z-position =',zpos)

    fig, ax2 = plt.subplots()
    ax2.imshow(rgb_proj)
   
    plt.show()

    return()
print('Slide the timeframe and z-position and select the desired channel to display: ')
interact(series, time=(0,imgsize[0]-1,1),zpos=(0,imgsize[2]-1,1), channel=channelname);

Slide the timeframe and z-position and select the desired channel to display: 


interactive(children=(IntSlider(value=15, description='time', max=31), IntSlider(value=0, description='zpos', …